### Usefule resources

https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-version.html

In [1]:
import os
import joblib
import sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import tarfile
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker.image_uris import retrieve

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


### Settings

In [5]:
bucket = 'sagemaker-bucket-ds'
base_dir = '/home/sagemaker-user/sagemaker/'
joblib_dir = 'models_joblib'
model_s3_path = '01_STOCKS/MODELS/model.tar.gz'

### Clients

In [14]:
s3 = boto3.client('s3')
sagemaker_client = boto3.client('sagemaker')

### Build the model

In [7]:
# Train and save the model
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
accuracy = model.score(X_test, y_test)
print(f"Model accuracy: {accuracy:.2f}")

Model accuracy: 1.00


### Save joblib file

In [10]:
model_dir = os.path.join(base_dir, 'JOBLIBS')
print(f"Model_dir: {model_dir}")
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, 'model.joblib')
joblib.dump(model, model_path)

Model_dir: /home/sagemaker-user/sagemaker/JOBLIBS


['/home/sagemaker-user/sagemaker/JOBLIBS/model.joblib']

### Create tar gz on S3

In [11]:
output_dir = os.path.join(base_dir, 'MODELS_TAR_GZ')
print(f"Output_dir: {output_dir}")
os.makedirs(output_dir, exist_ok=True)

# Define the full path for the tarball file
output_tar_path = os.path.join(output_dir, 'model.tar.gz')

# Package the model
with tarfile.open(output_tar_path, 'w:gz') as tar:
    tar.add(model_dir, arcname=os.path.basename(model_path))

Output_dir: /home/sagemaker-user/sagemaker/MODELS_TAR_GZ


### Upload tar gz to S3

In [13]:
s3.upload_file(output_tar_path, bucket, model_s3_path)

### Model settings

In [17]:
model_package_group_name = 'Proj1'
model_package_description = 'This is project1 (IRIS)'

region = 'eu-west-1'
framework = 'sklearn'
version = '1.2-1'

### Create model package group

In [21]:
try:
    # Create the model package group
    sagemaker_client.create_model_package_group(
        ModelPackageGroupName=model_package_group_name,
        ModelPackageGroupDescription=model_package_description
    )
    print(f"Model package group '{model_package_group_name}' created successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: An error occurred (ValidationException) when calling the CreateModelPackageGroup operation: Model Package Group already exists: arn:aws:sagemaker:eu-west-1:211125740051:model-package-group/proj1


In [22]:
model_url = f's3://{bucket}/{model_s3_path}'
role = get_execution_role()
image_uri = retrieve(framework, region, version=version)

In [23]:
model = Model(
    image_uri=image_uri,
    model_data=model_url,
    role=role,
    sagemaker_session=sagemaker.Session()
)


modelpackage_inference_specification = {
    "Containers": [
        {
            "Image": image_uri,
            "ModelDataUrl": model_url
        }
    ],
    "SupportedContentTypes": ["text/csv"],
    "SupportedResponseMIMETypes": ["text/csv"]
}

response = sagemaker_client.create_model_package(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageDescription='This is the description of the model',
    InferenceSpecification=modelpackage_inference_specification
)

print(response)

{'ModelPackageArn': 'arn:aws:sagemaker:eu-west-1:211125740051:model-package/Proj1/1', 'ResponseMetadata': {'RequestId': '5f99a395-1617-41b4-a7ac-4db0a234d8fd', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5f99a395-1617-41b4-a7ac-4db0a234d8fd', 'content-type': 'application/x-amz-json-1.1', 'content-length': '84', 'date': 'Wed, 19 Jun 2024 15:38:09 GMT'}, 'RetryAttempts': 0}}
